In [4]:
# import torch
import numpy as np
import data_pipes

df_path = "processed_aapl_data.csv"
data = data_pipes.process_df(df_path)

In [26]:
from torch import nn

class georgia_0(nn.Module):
    def __init__(self, win_past=20, features=19, dropout_rate=0.33):
        super().__init__()
        self.flat0 = nn.Flatten()

        self.hidden0 = nn.Linear(features*win_past, 128)
        self.act0 = nn.ReLU()
        self.dropout0 = nn.Dropout(dropout_rate)
        
        self.hidden1 = nn.Linear(128, 128)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.hidden2 = nn.Linear(128, 128)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.output = nn.Linear(128, 1)
    def forward(self, x):
        x = self.flat0(x)
        x = self.dropout0(self.act0(self.hidden0(x)))
        x = self.dropout1(self.act1(self.hidden1(x)))
        x = self.dropout2(self.act2(self.hidden2(x)))
        x = self.output(x)

        return x


In [36]:
import time
import torch
from torch import optim
import torch_ops as ops
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x, y = data["x"], data["y"]
v_split = 0.6
v_cutoff = int(len(x)*v_split)
x_t = torch.tensor(x[:v_cutoff], dtype=torch.float32).to(device)
x_v = torch.tensor(x[v_cutoff:], dtype=torch.float32).to(device)
y_t = torch.tensor(y[:v_cutoff], dtype=torch.float32).to(device)
y_v = torch.tensor(y[v_cutoff:], dtype=torch.float32).to(device)

win_past, features = x.shape[1], x.shape[2]
epochs, batch_size, lr, l2_decay = 250, 32, 2e-5, 1e-5

model = georgia_0(win_past=win_past, features=features).to(device)
loss_fn = nn.L1Loss()
optimizer = optim.Adam(params=model.parameters(), lr=lr)

loss = ops.test_loop(x_t, y_t, model, loss_fn, batch_size)
test_loss = ops.test_loop(x_v, y_v, model, loss_fn, batch_size)
print("---------- Epoch 0 ----------")
print(f"loss: {loss:1.4f}, test loss: {test_loss:1.4f}")

t0 = time.perf_counter()
for epoch in range(epochs):
    loss = ops.train_loop(x_t, y_t, model, loss_fn, optimizer, batch_size)
    test_loss_new = ops.test_loop(x_v, y_v, model, loss_fn, batch_size)

    if test_loss_new < test_loss or epoch == epochs - 1:
        print(f"---------- Epoch {epoch + 1} ----------")
        print(f"loss: {loss:1.4f}, test loss: {test_loss_new:1.4f}")
    
    improvement = test_loss_new < test_loss
    if improvement:
        test_loss = test_loss_new
        print(f"Checkpoint: test loss = {test_loss:1.4f} <---------------")
        torch.save(model.state_dict(), r"models/hanzo_0")

t1 = time.perf_counter()

print(f"Training done in {t1 - t0:1.2f} seconds.")

---------- Epoch 0 ----------
loss: 0.1744, test loss: 0.1658
---------- Epoch 1 ----------
loss: 0.1140, test loss: 0.1657
Checkpoint: test loss = 0.1657 <---------------
---------- Epoch 2 ----------
loss: 0.1177, test loss: 0.1656
Checkpoint: test loss = 0.1656 <---------------
---------- Epoch 3 ----------
loss: 0.1129, test loss: 0.1655
Checkpoint: test loss = 0.1655 <---------------
---------- Epoch 4 ----------
loss: 0.1094, test loss: 0.1654
Checkpoint: test loss = 0.1654 <---------------
---------- Epoch 5 ----------
loss: 0.1147, test loss: 0.1653
Checkpoint: test loss = 0.1653 <---------------
---------- Epoch 6 ----------
loss: 0.1130, test loss: 0.1652
Checkpoint: test loss = 0.1652 <---------------
---------- Epoch 7 ----------
loss: 0.1055, test loss: 0.1651
Checkpoint: test loss = 0.1651 <---------------
---------- Epoch 8 ----------
loss: 0.1187, test loss: 0.1651
Checkpoint: test loss = 0.1651 <---------------
---------- Epoch 9 ----------
loss: 0.1002, test loss: 0.1

In [22]:
x_t.dtype

dtype('float64')